## Project Requirements:
Scrape profiles and get:
1. the number of posts
2. the follower count
3. number of likes on most liked post
4. numbner of likes

In [1]:
from instagramy import InstagramUser
from instascrape import Profile, scrape_posts, Post
from selenium.webdriver import Chrome
from datetime import datetime
import pandas as pd
import numpy as np

In [2]:
chrome_driver_path = "/Users/chena23/Desktop/InstragramScraper/chromedriver"

driver = Chrome(chrome_driver_path)
headers = {
    "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Mobile Safari/537.36 Edg/87.0.664.57",
    "cookie": "sessionid=7320119797%3AsIWlEyDL37Q7Pg%3A14"
}

In [ ]:
def get_top_post (scraped_posts, cut_off_date = datetime(2021, 8,1)):
    # Checking if an empty array was inputed
    if (type(scraped_posts) != list):
        raise TypeError("Input is not a list")
    if (len(scraped_posts) == 0):
        raise ValueError("Empty posts list imported")
    
    #Returning Post with the highest number of likes
    top_post = scraped_posts[0]
    
    for post in scraped_posts:
        if (type(post) != Post):
            raise TypeError("One or more objects in the list is not a Post object")
        if (datetime.fromtimestamp(post.timestamp) < cut_off_date):
            if (post.likes > top_post.likes):
                top_post = post
        else:
            print("Post posted on {}, after the deadline".format(datetime.fromtimestamp(post.timestamp)))

    return top_post
        
    


In [ ]:
def check_valid_username (users):
    for i in range (0, len(users)):
        if (not pd.isnull(users.iloc[i]['IG Username'])):
            try:
                print(users.iloc[i]["IG Username"])
                profile = Profile(users.iloc[i]['IG Username'])
            except:
                print("{} is not a valid user name, the username is located at index {}, its name is {}".format(users.iloc[i]["IG Username"], i, users.iloc[i]['School']))

In [ ]:
users = pd.read_csv("Social_Media_Study_Handles.csv")
users = users.reindex(columns=[*users.columns, 'follower_count', 'number_of_posts', 'top_post_likes', 
'top_post_date_posted', 'top_post_url', 'first_post_likes', 'first_post_date_posted', 'first_post_url'])

#Specifying datatype for each columns
users['School'] = users['School'].astype(object)
users['IG Username'] = users['IG Username'].astype(object)
users['follower_count'] = users['follower_count'].astype('Int64')
users['number_of_posts'] = users['number_of_posts'].astype('Int64')
users['top_post_likes'] = users['top_post_likes'].astype('Int64')
users['top_post_date_posted'] = users['top_post_date_posted'].astype('datetime64[ns]')
users['top_post_url'] = users['top_post_url'].astype(object)
users['first_post_likes'] = users['first_post_likes'].astype('Int64')
users['first_post_date_posted'] = users['first_post_date_posted'].astype('datetime64[ns]')
users['first_post_url'] = users['first_post_url'].astype(object)

In [ ]:
users[users['IG Username'].isna()]

,School,IG Username,follower_count,number_of_posts,top_post_likes,top_post_date_posted,top_post_url,first_post_likes,first_post_date_posted,first_post_url
83,National Capital Consortium,NaN,<NA>,<NA>,<NA>,NaT,NaN,<NA>,NaT,NaN
89,Larkin,NaN,<NA>,<NA>,<NA>,NaT,NaN,<NA>,NaT,NaN
95,Icahn School of Medicine at Mount Sinai,NaN,<NA>,<NA>,<NA>,NaT,NaN,<NA>,NaT,NaN
100,Howard,NaN,<NA>,<NA>,<NA>,NaT,NaN,<NA>,NaT,NaN
104,University of South Carolina,NaN,<NA>,<NA>,<NA>,NaT,NaN,<NA>,NaT,NaN
105,San Antonio Uniformed Services Health Educatio...,NaN,<NA>,<NA>,<NA>,NaT,NaN,<NA>,NaT,NaN
108,BronxCare Health System,NaN,<NA>,<NA>,<NA>,NaT,NaN,<NA>,NaT,NaN
110,University of Arizona - Tucson (South Campus),NaN,<NA>,<NA>,<NA>,NaT,NaN,<NA>,NaT,NaN
112,University of Puerto Rico,NaN,<NA>,<NA>,<NA>,NaT,NaN,<NA>,NaT,NaN


In [ ]:
users.dtypes

School                            object
IG Username                       object
follower_count                     Int64
number_of_posts                    Int64
top_post_likes                     Int64
top_post_date_posted      datetime64[ns]
top_post_url                      object
first_post_likes                   Int64
first_post_date_posted    datetime64[ns]
first_post_url                    object
dtype: object

In [ ]:
print(not pd.isnull(users.iloc[89]['IG Username']))

False


In [ ]:
users.head(5).append(users.iloc[89])

,School,IG Username,follower_count,number_of_posts,top_post_likes,top_post_date_posted,top_post_url,first_post_likes,first_post_date_posted,first_post_url
0,TJU/Wills Eye Hospital,willseyeresidents,<NA>,<NA>,<NA>,NaT,NaN,<NA>,NaT,NaN
1,Bascom Palmer/University of Miami,bascompalmereye,<NA>,<NA>,<NA>,NaT,NaN,<NA>,NaT,NaN
2,Johns Hopkins/Wilmer,wilmereyeresidents,<NA>,<NA>,<NA>,NaT,NaN,<NA>,NaT,NaN
3,University of Iowa,uiowaeye,<NA>,<NA>,<NA>,NaT,NaN,<NA>,NaT,NaN
4,Mass. Eye and Ear,harvardophthalmologyresidents,<NA>,<NA>,<NA>,NaT,NaN,<NA>,NaT,NaN
89,Larkin,NaN,<NA>,<NA>,<NA>,NaT,NaN,<NA>,NaT,NaN


In [19]:
#test purposes
users = users.head(5).append(users.iloc[89])

for i in range (0, len(users)):
    if (not pd.isnull(users.iloc[i]['IG Username'])):
        #Scrape the profile
        print(users.iloc[i]['IG Username'])
        profile = Profile(users.iloc[i]['IG Username'])
        profile.scrape(headers=headers)

        #Adding profile datapoints to dataframe
        users.at[i, 'follower_count'] = profile.followers
        users.at[i, 'number_of_posts'] = profile.posts

        #Scraping Posts
        posts = profile.get_posts(webdriver=driver, login_first=True, login_pause=20)
        scraped_posts, unscraped = scrape_posts(posts, webdriver=driver, silent=False, headers=headers, pause=10)
        top_post = get_top_post(scraped_posts)

        #Post are scraped from most recent to oldest, therefore, the earliest post will be the last one scraped in theory
        first_post = scrape_posts[-1]

        users.at[i, 'top_post_likes'] = top_post.likes
        users.at[i, 'top_post_date_posted'] = datetime.fromtimestamp(top_post.timestamp)
        users.at[i, 'top_post_url'] = "instagram.com/p/{}".format(top_post.shortcode)
        users.at[i, 'first_post_likes'] = first_post.likes
        users.at[i, 'first_post_date_posted'] = datetime.fromtimestamp(first_post.timestamp)
        users.at[i, 'first_post_url'] = "instagram.com/p/{}".format(first_post.shortcode)

        

willseyeresidents
bascompalmereye
wilmereyeresidents
uiowaeye
harvardophthalmologyresidents


In [20]:
users

,School,IG Username,follower_count,number_of_posts,top_post_likes,top_post_date_posted,top_post_url,first_post_likes,first_post_date_posted,first_post_url
0,TJU/Wills Eye Hospital,willseyeresidents,1972,33,<NA>,NaT,NaN,<NA>,NaT,NaN
1,Bascom Palmer/University of Miami,bascompalmereye,12397,238,<NA>,NaT,NaN,<NA>,NaT,NaN
2,Johns Hopkins/Wilmer,wilmereyeresidents,1506,32,<NA>,NaT,NaN,<NA>,NaT,NaN
3,University of Iowa,uiowaeye,1468,129,<NA>,NaT,NaN,<NA>,NaT,NaN
4,Mass. Eye and Ear,harvardophthalmologyresidents,1542,179,<NA>,NaT,NaN,<NA>,NaT,NaN
89,Larkin,NaN,<NA>,<NA>,<NA>,NaT,NaN,<NA>,NaT,NaN


In [164]:
test_profile = Profile("willseyeresidents")
test_profile.scrape(headers=headers)

In [165]:
test_profile.posts

33

In [ ]:
posts = test_profile.get_posts(webdriver=driver, login_first=True, login_pause=20)

KeyboardInterrupt: 

In [ ]:
len(posts)

33

In [ ]:
scraped, unscraped = scrape_posts(posts, webdriver=driver, silent=False, headers=headers, pause=10)

0: CDABF2hhEhP - 2020-07-23 17:49:11
1: CC6_LwDhnTu - 2020-07-21 18:56:19
2: CC4Kk8ZBGIc - 2020-07-20 16:38:09
3: CCtc94uh_SW - 2020-07-16 12:47:12
4: CCqrp49B5j1 - 2020-07-15 10:57:49
5: CCjOYsGBwWO - 2020-07-12 13:27:23
6: CCecas4hdY- - 2020-07-10 16:53:47
7: CCHSdAehmf7 - 2020-07-01 17:04:11
8: CB3xPaehw43 - 2020-06-25 16:25:22
9: CBy5lzbBlq8 - 2020-06-23 19:02:08
10: CBwQp8qhq3y - 2020-06-22 18:25:57
11: CBtTXidBM4k - 2020-06-21 14:51:56
12: CBqW91ohOfD - 2020-06-20 11:25:39
13: CBoCRLXhQex - 2020-06-19 13:46:18
14: CBlvolqAn4t - 2020-06-18 16:25:00
15: CBjLzmHBJxE - 2020-06-17 16:33:27
16: CANgGIPAQc0 - 2020-05-15 09:56:03
17: CALFcpLAZq1 - 2020-05-14 11:24:43
18: B_lFRQnBz3j - 2020-04-29 17:12:01
19: B-sAg1yhXLQ - 2020-04-07 13:13:45
20: B9Z5wFmBtvD - 2020-03-06 14:56:56
21: B9HGomcBRkn - 2020-02-28 07:43:57
22: B8nbgg2hDsc - 2020-02-16 00:30:39
23: B7yyPaVB9wF - 2020-01-26 13:49:32
24: B6UcX8CB3J_ - 2019-12-20 22:29:51
25: B6TRZBrBHoW - 2019-12-20 11:34:38
26: B5l3vIoBB4d - 2019

In [ ]:
len(scraped)

33

In [ ]:
top_test_post = get_top_post(scraped)
print(top_test_post.likes, "instagram.com/p/{}".format(top_test_post.shortcode))

141 instagram.com/p/CC4Kk8ZBGIc
